In [102]:
import numpy as np
import pandas as pd
import json


In [103]:
credits= pd.read_csv("tmdb_5000_credits.csv")
movies= pd.read_csv("tmdb_5000_movies.csv")

In [104]:
movies.head(1)["production_companies"].values[0]

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [105]:
movies_details= movies.merge(credits, on='title')
movies_details.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [106]:
movies_details=movies_details[["title", "id", "overview", "genres", "keywords", "cast", "crew"]]

In [107]:
movies_details.dropna(inplace=True)

In [108]:
import ast
def convert(x):
    L=[]
    for item in  ast.literal_eval(x):
        L.append(item["name"])
    return L
movies_details["genres"]=movies_details["genres"].apply(convert)
movies_details["keywords"]=movies_details["keywords"].apply(convert)


In [109]:
def convert3(x):
    L=[]
    for item in  ast.literal_eval(x):
        L.append(item["name"])

    return L[0:3]
movies_details["cast"]=movies_details["cast"].apply(convert3)


In [110]:
def convert_director(x):
    L = []
    for item in  ast.literal_eval(x):
        if item["job"] == "Director":
            L.append(item["name"])
    return L
movies_details["crew"]=movies_details["crew"].apply(convert_director)

In [111]:
movies_details["overview"]=movies_details["overview"].apply(lambda x: x.split())

In [112]:
# movies_details["genres"]=movies_details["genres"].apply(lambda x: [item.replace(" ", "") for item in x])

In [113]:
def remove_space (x):
    L=[]
    for item in x:
        item=item.replace(" ", "")
        L.append(item)
    return L
movies_details["genres"]=movies_details["genres"].apply(remove_space) 
movies_details["keywords"]=movies_details["keywords"].apply(remove_space)   
movies_details["cast"]=movies_details["cast"].apply(remove_space)
movies_details["crew"]=movies_details["crew"].apply(remove_space)      

In [114]:
movies_details["tags"]=movies_details["overview"]+ movies_details["genres"]+movies_details["keywords"]+movies_details["cast"]+movies_details["crew"]

In [115]:
new_df=movies_details[["id", "title", "tags"]]
new_df

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [116]:
new_df["tags"]=new_df["tags"].apply(lambda x: " ".join(x))

/var/folders/3m/5_9nvcc55gd0cgb9ckkj_4hc0000gn/T/ipykernel_12406/1116665544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x: " ".join(x))


In [117]:
new_df["tags"]=new_df["tags"].apply(lambda x: x.lower())

/var/folders/3m/5_9nvcc55gd0cgb9ckkj_4hc0000gn/T/ipykernel_12406/445237715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x: x.lower())


In [118]:
from nltk.stem.porter import PorterStemmer  
ps=PorterStemmer()  
def stem(x):
    L=[]
    for item in x.split():
        L.append(ps.stem(item))
    return " ".join(L)    

new_df["tags"]=new_df["tags"].apply(stem)

/var/folders/3m/5_9nvcc55gd0cgb9ckkj_4hc0000gn/T/ipykernel_12406/2457799359.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(stem)


In [119]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000, stop_words="english")
vectors= cv.fit_transform(new_df["tags"]).toarray()
vectors.shape   


(4806, 5000)

In [120]:
from sklearn.metrics.pairwise import cosine_similarity
distance=cosine_similarity(vectors)


In [121]:
def recommend(movie):
    index=new_df[new_df["title"]==movie].index[0]
    recommended_movie=sorted(enumerate(distance[index]), reverse=True, key= lambda x: x[1])[1:6]
    for item in recommended_movie:
        print(new_df.iloc[item[0]]["title"])

In [122]:
recommend("Batman Begins")

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [123]:
import pickle 
pickle.dump(new_df.to_dict(), open("movie_dict.pkl", "wb"))

In [124]:
pickle.dump(distance,open("distance.pkl", "wb"))